In [1]:
import sys
import os
import numpy as np

from codingTools import *


In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
#originalFile="../images/03508649.1_512_512_2_0_12_0_0_0d.raw" #int16
originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
#ImageRaw = load_image_raw(originalFile, 1, 512, 512, "int16") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics = statistics(original_image)
entropy = statistics.compute_entropy()
print("order zero entropy of the image: ",entropy)

Max Value:  255
Min Value:  0
order zero entropy of the image:  7.426142695578908


In [4]:
######SAVE RAW IMAGE######
decodedFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0_copy.raw"
ImageRawDecoded = save_image_raw(decodedFile)
ImageRawDecoded.save_image_raw(original_image,"uint8")
###########

In [5]:
### Quantize ###
quantize = Quantizer(1, 10)
image_data_quantized = ImageRaw.get_image_structure_empty()
print(f"Imagen Original: {original_image}")
quantize.quantize(original_image, image_data_quantized)
print(f"Imagen quantizada: {image_data_quantized}")



Imagen Original: [[[123 123 118 ... 113 108 108]
  [125 124 122 ... 110 110 105]
  [124 123 123 ... 102 108 106]
  ...
  [130 131 132 ... 130 130 129]
  [129 129 128 ... 125 126 130]
  [129 127 129 ... 123 129 131]]]
Imagen quantizada: [[[12. 12. 11. ... 11. 10. 10.]
  [12. 12. 12. ... 11. 11. 10.]
  [12. 12. 12. ... 10. 10. 10.]
  ...
  [13. 13. 13. ... 13. 13. 12.]
  [12. 12. 12. ... 12. 12. 13.]
  [12. 12. 12. ... 12. 12. 13.]]]


In [6]:
### Dequantize ###
dequantize = Quantizer(1,10)
image_data_dequantized = ImageRaw.get_image_structure_empty()
dequantize.dequantize(image_data_quantized, image_data_dequantized)
print(f"Dequantize: {image_data_dequantized}")

Dequantize: [[[120. 120. 110. ... 110. 100. 100.]
  [120. 120. 120. ... 110. 110. 100.]
  [120. 120. 120. ... 100. 100. 100.]
  ...
  [130. 130. 130. ... 130. 130. 120.]
  [120. 120. 120. ... 120. 120. 130.]
  [120. 120. 120. ... 120. 120. 130.]]]


In [7]:
### PAE ###
pae = Metrics(original_image, image_data_dequantized)
print(pae.PAE_data)


9.0
9.0


In [8]:
###BITPACKING CODING###
# bitpacking_coding = BitPacking("bitpacket.txt" ,original_image)
# bitpacking_coding.coding()
# original_size_in_bytes = ImageRaw.get_original_size_in_bytes()
# CR = original_size_in_bytes / bitpacking_coding.size_of_coded_file
# print(original_size_in_bytes)
# print(bitpacking_coding.size_of_coded_file)